In [1]:
import os
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as tt
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2
from tqdm.notebook import tqdm
from torchvision.utils import save_image
from torchvision.utils import make_grid
import albumentations as A
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
DEVICE = torch.device('cuda')

In [3]:
os.chdir('/home/kotvkaske/Загрузки/FakeAVCeleb_v1.2/')

In [4]:
# from lib.data.dataset import *
# from lib.models.PFD import *
# from lib.utils import *
# from lib.training.train_model import *
# from lib.models.FTFD import *
# from lib.models.BaseDetector import *
# from lib.training.trainerold import train_old

In [5]:
os.listdir()

['FakeVideo-FakeAudio',
 'RealVideo-FakeAudio',
 'FakeVideo-RealAudio',
 'CUSTOM',
 'RealVideo-RealAudio',
 'meta_data.csv',
 'README.txt']

In [6]:
data = pd.read_csv('meta_data.csv')
data.rename(columns={'Unnamed: 9':'full_path'},inplace=True)
data['full_path'] = data['full_path'].apply(lambda x: '/'.join(x.split('/')[1:]))
data

,source,target1,target2,method,category,type,race,gender,path,full_path
0,id00076,-,-,real,A,RealVideo-RealAudio,African,men,00109.mp4,RealVideo-RealAudio/African/men/id00076
1,id00166,-,-,real,A,RealVideo-RealAudio,African,men,00010.mp4,RealVideo-RealAudio/African/men/id00166
2,id00173,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,RealVideo-RealAudio/African/men/id00173
3,id00366,-,-,real,A,RealVideo-RealAudio,African,men,00118.mp4,RealVideo-RealAudio/African/men/id00366
4,id00391,-,-,real,A,RealVideo-RealAudio,African,men,00052.mp4,RealVideo-RealAudio/African/men/id00391
...,...,...,...,...,...,...,...,...,...,...
21561,id07689,id06254,-,wav2lip,D,FakeVideo-FakeAudio,Asian (South),women,00028_id06254_wavtolip.mp4,FakeVideo-FakeAudio/Asian (South)/women/id07689
21562,id07689,id06343,-,wav2lip,D,FakeVideo-FakeAudio,Asian (South),women,00028_id06343_wavtolip.mp4,FakeVideo-FakeAudio/Asian (South)/women/id07689
21563,id07689,id07008,-,wav2lip,D,FakeVideo-FakeAudio,Asian (South),women,00028_id07008_wavtolip.mp4,FakeVideo-FakeAudio/Asian (South)/women/id07689
21564,id07689,id07377,-,wav2lip,D,FakeVideo-FakeAudio,Asian (South),women,00028_id07377_wavtolip.mp4,FakeVideo-FakeAudio/Asian (South)/women/id07689


In [7]:
_,audio,info = read_video('..FakeVideo-FakeAudio/African/men/id00076/00109_10_id00476_wavtolip.mp4')

NameError: name 'read_video' is not defined

### В итоге всего 500 униклаьных объектов (видео+аудио), из них все остальные получаются дф и тд.. Поэтому разбиваем на трейн/тест эти 500 объектов

In [50]:

for j in range(5):
    print(data.iloc[5])
    break

source                                       id00475
target1                                            -
target2                                            -
method                                          real
category                                           A
type                             RealVideo-RealAudio
race                                         African
gender                                           men
path                                       00099.mp4
full_path    RealVideo-RealAudio/African/men/id00475
Name: 5, dtype: object


In [46]:
data.sample(n=2)

,source,target1,target2,method,category,type,race,gender,path,full_path
13872,id01225,id01231,-,wav2lip,C,FakeVideo-RealAudio,Caucasian (American),women,00300_id01231_wavtolip.mp4,FakeVideo-RealAudio/Caucasian (American)/women...
19971,id00709,id01126,-,wav2lip,D,FakeVideo-FakeAudio,Caucasian (European),men,00206_id01126_wavtolip.mp4,FakeVideo-FakeAudio/Caucasian (European)/men/i...


In [ ]:
batch_size = 20
image_size = 224

In [ ]:
TRAIN = pd.read_csv('lib/data/FINAL_TRAIN_P2.csv')
TEST = pd.read_csv('lib/data/FINAL_TEST_P2.csv')
# TRAIN['path']=TRAIN['path'].apply(lambda x: 'lib/data/MixedData/'+x)
# TEST['path']=TEST['path'].apply(lambda x: 'lib/data/MixedData/'+x)
TRAIN['class'] = TRAIN['class'].apply(lambda x: torch.tensor(x,dtype=torch.float32))
TEST['class'] = TEST['class'].apply(lambda x: torch.tensor(x,dtype=torch.float32))

## Data Loading&Preparation

In [ ]:
def patch_apply(x):
    return torch.tensor([[x,x],[x,x]],dtype=torch.float32)

In [ ]:
TRAIN

,path,class
0,lib/data/MixedData/True/simswappic_107223.jpg,tensor(0.)
1,lib/data/MixedData/True/01687.png,tensor(0.)
2,lib/data/MixedData/Fake/newpic_977.jpgresult.jpg,tensor(1.)
3,lib/data/MixedData/Fake/3981.jpg,tensor(1.)
4,lib/data/MixedData/Fake/266.jpg,tensor(1.)
...,...,...
12271,lib/data/CUSTOMV2/Fake/id01637_1935__video1_au...,tensor(1.)
12272,lib/data/MixedData/Fake/swappicm_558.jpgresult...,tensor(1.)
12273,lib/data/MixedData/Fake/newpicm_735.jpgresult.jpg,tensor(1.)
12274,lib/data/MixedData/Fake/newpicm_598.jpgresult.jpg,tensor(1.)


In [ ]:
TRAIN['class'] = TRAIN['class'].apply(lambda x: patch_apply(x))
TEST['class'] = TEST['class'].apply(lambda x: patch_apply(x))
train_dataset = DeepFakeDataset(TRAIN)
test_dataset = DeepFakeDataset(TEST)

In [ ]:
TRAIN

,path,class
0,lib/data/MixedData/True/simswappic_107223.jpg,tensor(0.)
1,lib/data/MixedData/True/01687.png,tensor(0.)
2,lib/data/MixedData/Fake/newpic_977.jpgresult.jpg,tensor(1.)
3,lib/data/MixedData/Fake/3981.jpg,tensor(1.)
4,lib/data/MixedData/Fake/266.jpg,tensor(1.)
...,...,...
12271,lib/data/CUSTOMV2/Fake/id01637_1935__video1_au...,tensor(1.)
12272,lib/data/MixedData/Fake/swappicm_558.jpgresult...,tensor(1.)
12273,lib/data/MixedData/Fake/newpicm_735.jpgresult.jpg,tensor(1.)
12274,lib/data/MixedData/Fake/newpicm_598.jpgresult.jpg,tensor(1.)


In [ ]:
model = EffNetb1(patch=False,pretrained_folder = 'lib/models/b1_patch.pth')
model = model.to(DEVICE)
optimizer = torch.optim.Adam(model.parameters())
loss = torch.nn.BCELoss()
res = train_old(model,optimizer,loss,50,train_dataset,test_dataset,False,'lib/models/b1_default.pth')

In [ ]:
from lib.metrics.evaluate import Evaluator
checker = Evaluator(type_of_model='av',loss_fn=nn.BCELoss(),patch_v=True)
checker.estimate_dataset(test_dataset,model)

ValueError: too many values to unpack (expected 3)